<a href="https://colab.research.google.com/github/axionblast/gen-hw-set/blob/main/6.1_dpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Values Alignment
## Objectives
- Learn how to align a model's behavior using labelled preference data.


## Install and import necessary libraries  (~2 min)
### Ignore the warning if the blockes finish successfully.

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install peft
!pip install accelerate
!pip install tf-keras
!pip install numpy

In [ ]:
!pip install trl==0.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 11.6 MB/s eta 0:00:00


In [ ]:
import trl
print(trl.__version__)

0.8.6


In [ ]:
import os
import torch
import re
import json
import gdown
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer

## Load dataset

In [ ]:
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

Cloning into 'GenAI_hw6_dataset'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 4.06 KiB | 1.01 MiB/s, done.


In [ ]:
# Open and load the json dataset
with open("/content/GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("/content/GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

## Load model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'MediaTek-Research/Breeze-7B-Instruct-v0_1',
    device_map='auto',
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Get response from the original model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output)
    print('Response from original model:\n'+output+'\n')

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from original model:
真人化可能有助於提升日本漫畫行業的經濟效益，因真人版電影或劇集可以吸引更多觀眾，增加收入來源。然而，成功與否取決於作品的品質、行銷策略和市場接受度。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from original model:
真人化可能會影響日本漫畫原作者的創作動力，因真人版可能帶來新的靈感、挑戰，並吸引更多讀者。然而，個人感受和反應各異，有些作者可能因真人化而更投入創作，而其他人可能因個人喜好或對真人化看法不同而影響其動力。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from original model:
真人化可能會影響忠實的漫畫原作粉絲，因真人版可能有不同的故事改編、角色設定或表現方式，但個人喜好不同，仍有可能欣賞。

Question 8:
真

## Set parameters
### You only need to modify this block. Please don’t alter any other parts.

In [ ]:
num_epoch = 1
data_size = 10
support_ratio = 0

## Prepare training data

In [ ]:
# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 日本動漫真人化是否有損原作形象？ [/INST],真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。,真人化能夠呈現更真實的角色形象，提升原作魅力。
1,<s>回覆請少於20字 [INST] 真人化是否能夠擴大動漫在全球的影響力？ [/INST],真人化可能失去動漫的獨特風格，限制影響力擴大。,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。
2,<s>回覆請少於20字 [INST] 真人化是否能夠吸引新觀眾？ [/INST],真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。
3,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作故事情節的精髓？ [/INST],真人化可能因為改編而失去原作故事的深度與精髓。,真人化有機會更深入挖掘原作故事，保留精髓。
4,<s>回覆請少於20字 [INST] 真人化是否能夠提升動漫產業的商業價值？ [/INST],真人化可能讓觀眾對原作失去興趣，影響產業價值。,真人化能夠開拓更多商業機會，提升產業價值。
5,<s>回覆請少於20字 [INST] 真人化是否能夠保持原作的文化特色？ [/INST],真人化可能因為文化差異而失去原作獨有的文化魅力。,真人化可以透過場景、服裝等元素保留文化特色。
6,<s>回覆請少於20字 [INST] 真人化是否能夠挑戰技術上的新突破？ [/INST],真人化可能因為技術限制而無法達到動畫中的視覺效果。,真人化促使技術創新，挑戰視覺效果上的新高度。
7,<s>回覆請少於20字 [INST] 真人化是否會受到演員選擇的爭議？ [/INST],演員選擇可能引起爭議，觀眾難以接受角色塑造。,演員選擇可因應市場需求，不必受限於動畫形象。
8,<s>回覆請少於20字 [INST] 真人化是否能夠提高動漫的社會認同度？ [/INST],真人化可能因為劇情改編而無法贏得社會認同。,真人化有機會讓更多人接受動漫，提高社會認同度。
9,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作角色的個性特色？ [/INST],真人化可能因演員演技或導演選擇而失去角色的原有特色。,真人化可以透過演員表現保留角色的個性特色。


## Training

In [ ]:
training_args = TrainingArguments(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    report_to = 'none'
)

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [ ]:
print(train_dataset[0])

{'prompt': '<s>回覆請少於20字   [INST] 日本動漫真人化是否有損原作形象？ [/INST] ', 'chosen': '真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。', 'rejected': '真人化能夠呈現更真實的角色形象，提升原作魅力。'}


In [ ]:
dpo_trainer.train()

TypeError: DPOTrainer.get_batch_samples() takes 3 positional arguments but 4 were given

## Get response from the trained model

In [ ]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output)
    print('Response from trained model:\n'+output+'\n')

## Please observe the output of this block to complete your report, and don't forget to take a screenshot of the results

In [ ]:
model_response = []
print(f'num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}')
print()
for data in test_data:
    id = data['id']
    ref_output = original_model_response[id-1]
    output = trained_model_response[id-1]
    print(f'Question {id}:\n'+data['prompt'])
    print('Response from original model:\n'+ref_output)
    print('Response from trained model:\n'+output)
    print()
    model_response.append({'id':data['id'], 'prompt':data['prompt'], 'response_from_original_model':ref_output, 'response_from_trained_model':output})

## Get the output file

In [ ]:
with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
    json.dump(model_response, outfile, indent=4, ensure_ascii=False)